In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
control_chars = ['\x03', '\x11', '\x14', '\x15', '\x19', '\x1a', '\xe2', '\x80', '\x99t']
with open("/content/drive/MyDrive/NOLAAAAN.txt", 'r', encoding='utf-8') as f:
    text = f.read()
    for char in control_chars:
        text = text.replace(char, '')

In [ ]:
vocab_size = len(set(text))

In [ ]:
vocab_size

99

In [ ]:
set(text)

{'\n',
 ' ',
 '!',
 '"',
 '#',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '¯',
 'É',
 'Ö',
 'ä',
 'é',
 'ï',
 'ö',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”'}

In [ ]:
print(text[:1000])

BURNING.  Massive flames.  A dark shape emerges- The BAT
SYMBOL.  Growing.  Filling the screen with BLACKNESS.
CUT TO:: DAYLIGHT.  Moving over the towers of downtown Gotham...
Closing in on an office building... On a large window...
Which SHATTERS to reveal-

INT. OFFICE, HIGH RISE -- DAY

A man in a CLOWN MASK holding a SMOKING SILENCED PISTOL
ejects a shell casing.  This is DOPEY.  He turns to a second
man, HAPPY, also in clown mask, who steps forward with a
CABLE LAUNCHER, aims at a lower roof across the street and
FIRES a cable across.  Dopey secures the line to an I-beam
line- CLAMP on- sends a KIT BAG out then steps OUT the
window...

EXT. HIGH-RISE -- DAY

...into space.  The men SLIDE across the DIZZYING DROP...
landing on the lower roof across the street.

EXT. DOWNTOWN GOTHAM -- DAY

A MAN on the corner, back to us, holding a CLOWN MASK.  An
SUV pulls up.  The man gets in, puts on his mask.  Inside the
car- two other men wearing CLOWN MASKS.
GRUMPY: Three of a kind.  Let's do

In [ ]:
from minbpe import BPETokenizer
tokenizer = BPETokenizer()
tokenizer.train(text, vocab_size=1025)
# tokenizer.encode("hello world") # string -> tokens
# tokenizer.decode([1000, 2000, 3000]) # tokens -> string
tokenizer.save("/content/drive/MyDrive/NolEngine") # writes mymodel.model and mymodel.vocab


  0%|          | 0/769 [00:00<?, ?it/s]

In [ ]:
from minbpe import BPETokenizer
tokenizer = BPETokenizer()a
tokenizer.load("/content/drive/MyDrive/NolEngine.model") # loads the model back, the vocab is just for vis

In [ ]:
tokened_text = tokenizer.encode(text)

In [ ]:
import pickle

with open('/content/drive/MyDrive/tokened_text', 'wb') as fp:
    pickle.dump(tokened_text, fp)


In [ ]:
with open ('/content/drive/MyDrive/tokened_text', 'rb') as fp:
    itemlist = pickle.load(fp)

True

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.notebook import trange


# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 1000
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab_size = 1025
# ------------

torch.manual_seed(1337)

# Train and test splits
data = torch.tensor(tokened_text, dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in trange(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model


0.335617 M parameters


  0%|          | 0/10000 [00:00<?, ?it/s]

step 0: train loss 7.0991, val loss 7.1043
step 1000: train loss 4.3791, val loss 4.5881
step 2000: train loss 4.0588, val loss 4.3475
step 3000: train loss 3.8608, val loss 4.2068
step 4000: train loss 3.6864, val loss 4.0685
step 5000: train loss 3.5532, val loss 4.0092
step 6000: train loss 3.4795, val loss 3.9577
step 7000: train loss 3.3971, val loss 3.9064
step 8000: train loss 3.3480, val loss 3.8703
step 9000: train loss 3.3111, val loss 3.8760
step 9999: train loss 3.2606, val loss 3.8320


In [ ]:
tokenizer.encode("Batman")

[616, 598]

In [ ]:
context = torch.tensor([[616, 598]], dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Batman
other light off with Truman, Theatre when ANGIER shalleforRachel, STARING at the cro into a lowrifists...

INT. HELEZE WDERVEL OF MOMENTS LATER

Your‘9:

INT. ROOM 2022 SW and WITHOUSE.:

INT. HIRSTIPLEY OFFICE, SOLDENT -- CONTINUOUS

SauThe TV as CLOOMANS, there. He hears outs the balcone see a house
on, from Makes his around MALLEDS his armenation funcayault-hooked bunscrial. An I
arballals.'Cutter reaching vehiclap-
Fary. Moving off the head bar.
The Batmobile – the window, easylying.  Batman
pressurrow him, Angier is a plane, unconscience.
How won’t had the worlder.
RICHIA: Oh, Mr.Volsed. I was security, Alone
him are, not her-
OPPENHEIMER (V.O.): You’re dissepect.
You ypoint myself.
141  INT. LEONARD’S APARTS as Tach OFFICE -- DAY (COLOUR)

BLAKENCTS from the ircius.
BORDEN: That’s whatever, WHACHES INDING at the world.
MAY: You gam’s starts –
PROTAGONIST: Can you know. Bruce Dentle.
RACHEL: This isn’t your
than. I mo happender,                   *
you’re gonna panlike anim

In [ ]:
for iter in trange(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 0/10000 [00:00<?, ?it/s]

step 0: train loss 3.2599, val loss 3.8406
step 1000: train loss 3.2262, val loss 3.8110
step 2000: train loss 3.2029, val loss 3.8078
step 3000: train loss 3.1587, val loss 3.8168
step 4000: train loss 3.1449, val loss 3.8028
step 5000: train loss 3.1139, val loss 3.7756
step 6000: train loss 3.1122, val loss 3.7672
step 7000: train loss 3.0929, val loss 3.7794
step 8000: train loss 3.0619, val loss 3.7749
step 9000: train loss 3.0716, val loss 3.7468
step 9999: train loss 3.0501, val loss 3.7500


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/NolEngine1.pt")

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import trange
import math

# Enhanced hyperparameters
batch_size = 32  # increased for better gradient estimates
block_size = 128  # increased context length
max_iters = 20000
eval_interval = 500
learning_rate = 3e-4  # slightly lower for better stability
warmup_iters = 1000  # learning rate warmup
min_lr = 1e-5  # learning rate floor
weight_decay = 0.1  # L2 regularization
grad_clip = 1.0  # gradient clipping
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128  # increased embedding dimension
n_head = 8    # increased number of heads
n_layer = 6   # increased number of layers
dropout = 0.1 # added dropout for regularization
vocab_size = 1025

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias """
    def __init__(self, ndim, bias=False):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

class MultiHeadAttention(nn.Module):
    """ Multi-head self-attention with improved efficiency """

    def __init__(self, n_embd, n_head):
        super().__init__()
        assert n_embd % n_head == 0

        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=False)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        # causal mask
        self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size))
                                    .view(1, 1, block_size, block_size))
        self.n_head = n_head
        self.n_embd = n_embd

    def forward(self, x):
        B, T, C = x.shape

        # calculate query, key, values for all heads in batch
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class FeedForward(nn.Module):
    """ Enhanced feedforward network with GELU activation """

    def __init__(self, n_embd):
        super().__init__()
        self.c_fc    = nn.Linear(n_embd, 4 * n_embd, bias=False)
        self.c_proj  = nn.Linear(4 * n_embd, n_embd, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.gelu    = nn.GELU()

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):
    """ Transformer block with improved architecture """

    def __init__(self, n_embd, n_head):
        super().__init__()
        self.ln_1 = LayerNorm(n_embd, bias=False)
        self.attn = MultiHeadAttention(n_embd, n_head)
        self.ln_2 = LayerNorm(n_embd, bias=False)
        self.ffwd = FeedForward(n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.ffwd(self.ln_2(x))
        return x

class ImprovedLanguageModel(nn.Module):
    """ Enhanced language model with improved architecture and training stability """

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = LayerNorm(n_embd, bias=False)
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

        # Initialize weights
        self.apply(self._init_weights)
        # Apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """Enhanced generation with temperature and top-k sampling"""
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = float('-inf')

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

def get_lr(it):
    # Learning rate schedule: linear warmup and cosine decay
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    decay_ratio = (it - warmup_iters) / (max_iters - warmup_iters)
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (learning_rate - min_lr)

# Training loop
model = ImprovedLanguageModel().to(device)
print(f'Number of parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for iter in trange(max_iters):
    # Learning rate schedule
    lr = get_lr(iter)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    clip_grad_norm_(model.parameters(), grad_clip)
    optimizer.step()

Number of parameters: 1.46M


  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=100, temperature=0.7, top_k=50)

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/NolEngine_Claude1.pt")

In [ ]:
tokenizer.decode(generated[0].tolist())

'\x00GROVES the sip of the GOTHAM OF STATION.\nSTEWARD: Where’re the enemy?\nSTRAUSS: I’m about a child day.\nSENATE AIDE: What’s that?\nSENATE AIDE: We’re a lifect. Just execretation,\nservicious.\nSTRAUSS: Have prosec'

In [ ]:
generated

tensor([[  0,  71, 358,  86, 468, 278, 383, 778, 482,  71,  79, 367, 537,  32,
         669,  32, 337, 334, 893, 262, 337,  69,  87, 500, 267,  87, 407, 101,
         668, 264, 276, 908, 121, 327, 337, 421, 759, 977, 526, 109, 286,  98,
         397, 315,  99, 283, 460, 100, 357, 645, 320, 334, 306,  65, 873, 402,
         447, 288, 355, 745, 327,  83, 320, 334, 306,  65, 873, 402, 825, 668,
         315, 338, 603,  99, 116, 271,  74, 574, 596, 651, 287, 116, 784, 469,
         115, 263, 118, 372, 105, 265, 431, 337, 421, 759, 977,  72,  97, 347,
         744, 312,  99]], device='cuda:0')

In [ ]:
context = torch.tensor([[616, 598]], dtype=torch.long, device=device)
print(tokenizer.decode(model.generate(context, max_new_tokens=1000, temperature=0.7, top_k=50)[0].tolist()))

Batman, the "Body
break STARGE to the ground, thoughtful to find Martha calls around
DIMSELY and we see a fenciling in
a new mirror woman is SMOKING...
10.
BATMAN: What was it?
DENT: They’re a couple of time - you can
find any to find Man.
Maybe he showed me the men -
78.
GORDON: I’m sorry, you don’t know what your
real hole, then?
Ready nods. Gordon nods. Nods.
RACHEL: Dr Mann’s finity, sir. She killed.
RACHEL: To Harvey?! I am given the
Party of us with Dr Oppenheimer!
(climbs in, cloudsport)
I told you that dropping to
say to your plan?
RACHEL: And I don’t remember this guy, I never say it,
I’ll think it was a mistake.
Down bed, hoping to take it after you. Endust
yourself and sname, she was worked before.
You might be untired with the other man’re
supposed over for a dressing manner.
But the bomb where you are, you’ll have to do
anything?
RACHEL: Maybe I’m not here for you all
wondered for against children.
RA’S AL GHUL (CONT’D)
To frecond?
BATMAN: If he has no charge for that.
Bru

In [ ]:
model = torch.load("/content/drive/MyDrive/NolEngine_Claude.pt")

<ipython-input-45-b9f988c7627a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/content/drive/MyDrive/NolEngine_Claude.pt")


In [ ]:
for iter in trange(max_iters):
    # Learning rate schedule
    lr = get_lr(iter)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    clip_grad_norm_(model.parameters(), grad_clip)
    optimizer.step()

  0%|          | 0/20000 [00:00<?, ?it/s]

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
model